### MERGING AND FILTERING .TXT FILES USING SPARK

In [ ]:
sqlContext = SQLContext(sc)
import pandas as pd
import os
import matplotlib
matplotlib.use('Agg') 
import seaborn as sns
%matplotlib inline
from pyspark.sql.types import *

i=0
list_files = []
schema = StructType([])
df1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)
df1_ocurrences = sqlContext.createDataFrame(sc.emptyRDD(), schema)
df1_count = sqlContext.createDataFrame(sc.emptyRDD(), schema)
df1_merged = sqlContext.createDataFrame(sc.emptyRDD(), schema)
df1_complete = sqlContext.createDataFrame(sc.emptyRDD(), schema)

sc.setCheckpointDir('/resources/data/checkpoints/')

def map_func (dados):

    
    df1 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("/resources/data/" + dados + ".csv")
    
    df1.rdd.checkpoint()
    
    # AGGREGATION
    df1 = df1.withColumn("Interval", ((df1.Timestamp.cast("long") / 1).cast("long") * 1).cast("timestamp")) 
    # COUNT 1s
    df1_ocurrences = df1.groupBy("Interval").sum("Bit_value")
    df1_ocurrences = df1_ocurrences.withColumnRenamed("sum(Bit_value)", "Sum_df1")
    # COUNT TOTAL
    df1_count = df1.groupBy("Interval").count()
    df1_count = df1_count.withColumnRenamed("count", "Total_df1")
    # MERGING
    df1_merged = df1_ocurrences.join(df1_count, ["Interval"])

    df1_complete = df1_merged.withColumn(str(dados), df1_merged.Sum_df1 / df1_merged.Total_df1)
    df1_complete = df1_complete.drop('Sum_df1')
    df1_complete = df1_complete.drop('Total_df1')
    
    df1_complete.rdd.checkpoint()
    df1_complete = sqlContext.createDataFrame(df1_complete.rdd, df1_complete.schema)
    
    return df1_complete

def joining(df1, df2):
#     df1.rdd.checkpoint()

    df1 = df1.join(df2,['Interval']) 

    return df1


for file in os.listdir('/resources/data/'):
    var_name = file.split(".")[0]
    
    list_files.append(var_name)

#range = list_files
dataframes = map(lambda r: map_func(r) ,list_files)
print ('Mapper complete')

#data = reduce(lambda a, b: a.join(b,['Interval']),dataframes)
data = reduce(lambda a, b: joining(a,b), dataframes)
print('Reducer complete')